# 1) Large size vectors: lcc MAD-Mex

## a) Normalize, reproject to wgs84

In [ ]:
import os

In [2]:
cur_directory = os.getcwd()

In [3]:
dir_alg_python = '/LUSTRE/MADMEX/tasks/2020/6_geonode/geonode-issue-3/python3_package_for_geonode/geonode_conabio'

In [4]:
os.chdir(dir_alg_python)

In [5]:
import fiona
from pyproj import Proj
from fiona.crs import to_string

from wrappers import reproj_normalize_and_write_large_vector


In [7]:
#direc = "/LUSTRE/MADMEX/products/landcoverchange/landsat/mosaics/2017_2018"
direc = "/LUSTRE/MADMEX/products/landcoverchange/sentinel2/2017_2018/indi50k/estados/HIDALGO/"

In [8]:
#input_filename = direc + "/madmex_landsat_changes_2017-2018.shp"
#output_filename = direc + "/madmex_landsat_changes_2017-2018_wgs84_using_fiona.shp"
#layer = "madmex_landsat_changes_2017-2018_wgs84_using_fiona"
input_filename = direc + "HIDALGO_merge.shp"
output_filename = direc + "HIDALGO_merge_wgs84_using_fiona.shp"
layer = "HIDALGO_merge_wgs84_using_fiona"
#dst_crs = "4326"
name_attribute1 = "t1_dsc_31"
name_attribute2 = "t2_dsc_31"
name_attribute3 = "t1_dsc_17"
name_attribute4 = "t2_dsc_17"
name_attribute5 = "cmb_dsc_31"
name_attribute6 = "cmb_dsc_17"

In [9]:
list_name_attributes = [name_attribute1, name_attribute2,
                        name_attribute3, name_attribute4,
                        name_attribute5, name_attribute6]

**Read and call wrapper**

In [13]:
with fiona.open(input_filename) as src:
    fc = (feature for feature in src if not None in [feature['properties'][att] for att in list_name_attributes] and
                                                     feature['geometry'] is not None)
    src_crs = to_string(src.crs)
    proj_src_crs = Proj(src_crs)
    fc_schema = src.schema
    if not proj_src_crs.crs.is_geographic:
        reproj_normalize_and_write_large_vector(fc, fc_schema,
                                                list_name_attributes,
                                                layer,
                                                output_filename,
                                                src_crs, 
                                                is_geographic = False
                                                )
    else:
        reproj_normalize_and_write_large_vector(fc, fc_schema, 
                                                list_name_attributes,
                                                layer,
                                                output_filename,
                                                src_crs
                                                )
        

## b) Register large vector layer in geonode via `shp2pgsql` and `updatelayers`

**Make sure stack deployment of geonode is up and running**

**1) Create `~/.pgpass` with contents: `hostname:port:database:username:password` and `chmod 0600 ~/.pgpass`**

In [9]:
from utils_cli import create_table_via_shp2pgsql

In [10]:
#next need to be in lowercase
name_table = "hidalgo_madmex_lcc_2017_2018_fiona" #madmex_landsat_2017-2018_lcc


In [24]:
[result_create_table, name_table, out, err] = create_table_via_shp2pgsql(output_filename, name_table)

hidalgo_madmex_lcc_2017_2018_fiona
after wait


In [25]:
print((result_create_table, out, err))

(0, None, b'')


**2) Add to geoserver from geonode_data database**. Need to follow: https://training.geonode.geo-solutions.it/004_admin_workshop/007_loading_data_into_geonode/geoserver.html

In [1]:
from docker import APIClient


In [2]:
c = APIClient(base_url='unix://var/run/docker.sock')

In [3]:
string1 = "from geoserver.catalog import Catalog;"


In [4]:
string2 = "cat = Catalog('http://geonode.conabio.gob.mx' + '/geoserver/rest', \'"

In [5]:
user = "miuser"

In [6]:
passwd = "mipassword"

In [8]:
string3 = "gstore = cat.get_stores(names=['geonode_data'], workspaces=['geonode']);"

In [9]:
epsg_code = "EPSG:4326"

In [10]:
native_name = "aguascalientes_merge_wgs84_fiona"

In [11]:
string4 = "cat.publish_featuretype(native_name, gstore[0], epsg_code, native_name=native_name)"

In [12]:
string = "".join([string, string2, 
                  user, "\', ", "\'", passwd, "\');",
                  string3,
                  "native_name = \'", native_name, "\';", 
                  "epsg_code = \'", epsg_code, "\';", 
                  string4])

In [14]:
cmd = "".join(["python -c ", "\"", string, "\""])

In [16]:
ex = c.exec_create(container = 'spcgeonode_django_1', 
                   cmd = cmd)


In [17]:
ex_start = c.exec_start(exec_id=ex)

In [18]:
print(ex_start)

b'Traceback (most recent call last):\n  File "<string>", line 1, in <module>\n  File "/usr/local/lib/python3.7/site-packages/geoserver/catalog.py", line 889, in publish_featuretype\n    raise FailedRequestError(\'Failed to publish feature type {} : {}, {}\'.format(name, resp.status_code, resp.text))\ngeoserver.catalog.FailedRequestError: Failed to publish feature type aguascalientes_merge_wgs84_fiona : 500, Resource named \'aguascalientes_merge_wgs84_fiona\' already exists in store: \'geonode_data\'\n'


**3) Update layers using docker container `spcgeonode_django_1`**

In [11]:
from utils_docker import update_layers_via_docker

In [12]:
result_update = update_layers_via_docker(name_table)

In [14]:
#print(result_update)

**4) Make sure you are able to download it and see thumbnail. If not click to button refresh attributes and statistics for the layer in geonode. For thumbnail increase nginx conf proxy_read_timeout parameter.**

# 2) Small or medium size raster: lc MAD-Mex

## a) Reproject to wgs84, compress and tile it

Based on: 

https://rasterio.readthedocs.io/en/latest/topics/reproject.html

https://rasterio.readthedocs.io/en/latest/topics/image_options.html

In [1]:
from pyproj import Proj
import rasterio

from geonode_conabio.wrappers import reproj_and_write_one_band_raster

In [2]:
#direc = '/LUSTRE/MADMEX/products/landcover/sentinel2/2017/31'
#direc = '/LUSTRE/MADMEX/products/landcover/sentinel2/2017/estados/Aguascalientes/31/'
#direc = '/LUSTRE/MADMEX/products/landcover/sentinel2/2017/estados/Hidalgo/31/'
direc = '/LUSTRE/MADMEX/products/landcover/landsat/2015/32/'

In [4]:
#input_filename = ''.join([direc, '/madmex_sentinel2_2017_31.tif'])
#output_filename = ''.join([direc, '/madmex_sentinel2_2017_31_wgs84_tiled_rasterio.tif'])
#input_filename = ''.join([direc, '/Aguascalientes_2017_31.tif'])
#output_filename = ''.join([direc, '/Aguascalientes_2017_31_wgs84_tiled_rasterio.tif'])
#input_filename = ''.join([direc, '/Hidalgo_2017_31.tif'])
#output_filename = ''.join([direc, '/Hidalgo_2017_31_wgs84_tiled_rasterio.tif'])
#input_filename = "".join([direc, "/eoss4tnc_landcover_landsat8_2016_20200220_auxlabel_madmexplus_QRO.tif"])
#output_filename = "".join([direc, "/eoss4tnc_landcover_landsat8_2016_20200220_auxlabel_madmexplus_QRO_rasterio.tif"])
input_filename = "".join([direc, "madmex_lc_2015.tif"])
output_filename = "".join([direc, "madmex_lc_2015_wgs84_tiled_rasterio.tif"])
#dst_crs = 'EPSG:4326'

In [5]:
with rasterio.open(input_filename) as src:
    src_crs = src.crs.to_string()
    proj_crs = Proj(src_crs)
    if not proj_crs.crs.is_geographic:
        reproj_and_write_one_band_raster(src, output_filename,
                                         is_geographic=False)
    else:
        reproj_and_write_one_band_raster(src, output_filename)

## b) Register small, medium or large raster layer in geonode via `importlayers`

**1) Importlayers cmd**

In [1]:
direc = '/LUSTRE/MADMEX/products/landcover/landsat/2015/32/'

In [2]:
output_filename = "".join([direc, "madmex_lc_2015_wgs84_tiled_rasterio.tif"])

In [3]:
region = "Mexico, North America, Latin America"
name="National_MAD-Mex_landsat8_lc_2015_32_classes"
title="National land cover map 2015 32 classes"
kw="MAD-Mex, Landsat8, GeoTIFF, WCS"
abstract="Land cover sourced by Landsat8 satellite images. All available images for the required calendar years were acquired through the ESPA processing system and were provided as atmospherically corrected surface reflectance and are accompanied by quality layers assigning, amongst others, pixels obscured by clouds and cloud shadows. Land cover is generated using a novel semi automated workflow based on time series processing and optimized training data generation. The workflow consists of the following procedures, including data acquisition, time series generation and processing, spectral clustering, training data generation and optimization, supervised object-based classifications, and geometry harmonization. "


In [4]:
region = ''.join(["\"", region, "\""])
name = ''.join(["\"", name, "\""])
title = ''.join(["\"", title, "\""])
abstract = ''.join(["\"", abstract, "\""])
kw = ''.join(["\"", kw, "\""])

In [5]:
from geonode_conabio.utils_docker import import_layers_via_docker

/LUSTRE/MADMEX/tasks/2020/6_geonode/geonode-master/python3_package_for_geonode/geonode_conabio/settings.py:12: UserWarning: No configuration file found in /root/.geonode_conabio, some functionalities won't be available
  warnings.warn('No configuration file found in %s, some functionalities won\'t be available' % dotenv_path)


In [6]:
result_import = import_layers_via_docker(region, name, title,
                                         abstract, kw,
                                         output_filename
                                         )

In [14]:
#print(result_import)

**2) Make sure you are able to download it and see thumbnail. If not click to button refresh attributes and statistics for the layer in geonode. For thumbnail increase nginx conf proxy_read_timeout parameter.**

# 2) Large size raster: lc MAD-Mex

## a) Reproject to wgs84, compress and tile it

In [1]:
from pyproj import Proj
import rasterio

from geonode_conabio.wrappers import reproj_and_write_one_band_raster

In [2]:
#direc = '/LUSTRE/MADMEX/products/landcover/sentinel2/2017/31'
#direc = '/LUSTRE/MADMEX/products/landcover/sentinel2/2017/estados/Aguascalientes/31/'
#direc = '/LUSTRE/MADMEX/products/landcover/sentinel2/2017/estados/Hidalgo/31/'
direc = '/LUSTRE/MADMEX/products/landcover/landsat/2015/32/'

In [4]:
#input_filename = ''.join([direc, '/madmex_sentinel2_2017_31.tif'])
#output_filename = ''.join([direc, '/madmex_sentinel2_2017_31_wgs84_tiled_rasterio.tif'])
#input_filename = ''.join([direc, '/Aguascalientes_2017_31.tif'])
#output_filename = ''.join([direc, '/Aguascalientes_2017_31_wgs84_tiled_rasterio.tif'])
#input_filename = ''.join([direc, '/Hidalgo_2017_31.tif'])
#output_filename = ''.join([direc, '/Hidalgo_2017_31_wgs84_tiled_rasterio.tif'])
#input_filename = "".join([direc, "/eoss4tnc_landcover_landsat8_2016_20200220_auxlabel_madmexplus_QRO.tif"])
#output_filename = "".join([direc, "/eoss4tnc_landcover_landsat8_2016_20200220_auxlabel_madmexplus_QRO_rasterio.tif"])
input_filename = "".join([direc, "madmex_lc_2015.tif"])
output_filename = "".join([direc, "madmex_lc_2015_wgs84_tiled_rasterio.tif"])
#dst_crs = 'EPSG:4326'

In [5]:
with rasterio.open(input_filename) as src:
    src_crs = src.crs.to_string()
    proj_crs = Proj(src_crs)
    if not proj_crs.crs.is_geographic:
        reproj_and_write_one_band_raster(src, output_filename,
                                         is_geographic=False)
    else:
        reproj_and_write_one_band_raster(src, output_filename)

## b) Register large raster layer in geonode via `importlayers`

**1) Importlayers cmd**

In [1]:
direc = '/LUSTRE/MADMEX/products/landcover/landsat/2015/32/'

In [2]:
output_filename = "".join([direc, "madmex_lc_2015_wgs84_tiled_rasterio.tif"])

In [3]:
region = "Mexico, North America, Latin America"
name="National_MAD-Mex_landsat8_lc_2015_32_classes"
title="National land cover map 2015 32 classes"
kw="MAD-Mex, Landsat8, GeoTIFF, WCS"
abstract="Land cover sourced by Landsat8 satellite images. All available images for the required calendar years were acquired through the ESPA processing system and were provided as atmospherically corrected surface reflectance and are accompanied by quality layers assigning, amongst others, pixels obscured by clouds and cloud shadows. Land cover is generated using a novel semi automated workflow based on time series processing and optimized training data generation. The workflow consists of the following procedures, including data acquisition, time series generation and processing, spectral clustering, training data generation and optimization, supervised object-based classifications, and geometry harmonization. "


In [4]:
region = ''.join(["\"", region, "\""])
name = ''.join(["\"", name, "\""])
title = ''.join(["\"", title, "\""])
abstract = ''.join(["\"", abstract, "\""])
kw = ''.join(["\"", kw, "\""])

In [5]:
from geonode_conabio.utils_docker import import_layers_via_docker

/LUSTRE/MADMEX/tasks/2020/6_geonode/geonode-master/python3_package_for_geonode/geonode_conabio/settings.py:12: UserWarning: No configuration file found in /root/.geonode_conabio, some functionalities won't be available
  warnings.warn('No configuration file found in %s, some functionalities won\'t be available' % dotenv_path)


In [6]:
result_import = import_layers_via_docker(region, name, title,
                                         abstract, kw,
                                         output_filename
                                         )

In [14]:
#print(result_import)

**2) Make sure you are able to download it and see thumbnail. If not click to button refresh attributes and statistics for the layer in geonode. For thumbnail increase nginx conf proxy_read_timeout parameter.**

**3) Pack tif and style into zip and copy to path for ftp**

```
/shared_volume/geonode/scripts/spcgeonode/_volume_geodatadir/workspaces/geonode/styles/madmex_32_classes.sld

/shared_volume/geonode/scripts/spcgeonode/_volume_geodatadir/workspaces/geonode/styles/madmex_31_classes.sld

/shared_volume/geonode/scripts/spcgeonode/_volume_geodatadir/data/geonode/Test3_National_MAD-Mex_landsat8_lc_2015_32_classes_512_512/Test3_National_MAD-Mex_landsat8_lc_2015_32_classes_512_512.geotiff

```

In [1]:
from docker import APIClient

In [2]:
c = APIClient(base_url='tcp://172.17.0.1:1111')
#c = APIClient(base_url='unix://var/run/docker.sock')


In [3]:
title_layer = "Test3 National land cover map 2015 32 classes 512 512" #this will be input of function
destiny_dir = "/shared_volume"

In [4]:
string1 = """import os;
import django;
os.chdir('/spcgeonode');
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'geonode.local_settings');
django.setup();
from geonode.base.models import Link;
from geonode.layers.models import Layer;
"""

In [5]:
string2 = "layer = Layer.objects.filter(title="
                                         
string3 = ").first();"

In [6]:
string4 = "style_name = layer.default_style.name;"

string5 = "layer_name = layer.name;"

#string6 = 

In [7]:
string = "".join([string1, 
                  "title_layer = \'", title_layer, "\';",
                  string2, "\'",
                  title_layer, "\'", 
                  string3,
                  string4,
                  string5,
                  "print(layer_name);",
                  "print(style_name)"])

In [8]:
cmd = "".join(["python -c ", "\"", string, "\""])

In [9]:
ex = c.exec_create(container = 'spcgeonode_django_1', 
                   cmd = cmd)
ex_start = c.exec_start(exec_id=ex) #if arg stream=True in exec_start method, then returns generator

print(ex_start)

b'Test3_National_MAD-Mex_landsat8_lc_2015_32_classes_512_512\nmadmex_32_classes\n'


In [10]:
import os
import shutil

In [11]:
ex_start.decode().split("\n")[0:2]

['Test3_National_MAD-Mex_landsat8_lc_2015_32_classes_512_512',
 'madmex_32_classes']

In [12]:
layer_name, style_name = ex_start.decode().split("\n")[0:2]

In [13]:
layer_name

'Test3_National_MAD-Mex_landsat8_lc_2015_32_classes_512_512'

In [14]:
style_name

'madmex_32_classes'

In [15]:
destiny_path = "/shared_volume/ftp_dir/"
style_source_path = "/shared_volume/geonode/scripts/spcgeonode/_volume_geodatadir/workspaces/geonode/styles/" + style_name + ".sld"
layer_source_path = "/shared_volume/geonode/scripts/spcgeonode/_volume_geodatadir/data/geonode/" + layer_name + "/" + layer_name + ".geotiff"

**Read sld and write cmap to raster**

References:

https://github.com/CONABIO/antares3-sandbox/blob/master/notebooks/forests_maps_julian/model_predict_and_db_to_raster_pipeline.ipynb

https://github.com/CONABIO/antares3/blob/develop/madmex/util/db.py

https://github.com/CONABIO/antares3-sandbox/blob/master/notebooks/rapideye_parsing_metadaata/metadata_parser.ipynb

In [17]:
import rasterio
import xml.etree.ElementTree as ET

In [20]:
root = ET.parse(style_source_path).getroot()

In [21]:
ns = "http://www.opengis.net/sld"

In [22]:
rf = root.find('ns:UserLayer/ns:UserStyle/ns:FeatureTypeStyle/ns:Rule/ns:RasterSymbolizer/ns:ColorMap',namespaces={'ns': ns})

In [23]:
rf_children = rf.getchildren()

In [24]:
def hex_to_rgba(hex_code):
    return tuple(int(hex_code[i:i+2], 16) for i in (1, 3 ,5)) + (255,)

In [25]:
hex_dict = dict([(i.attrib["quantity"], i.attrib["color"]) for i in rf_children])

In [26]:
rgb_dict = {k:hex_to_rgba(v) for k,v in hex_dict.items()}
rgb_dict[0] = (0,0,0,0)

In [27]:
rgb_dict

{'1': (0, 81, 0, 255),
 '2': (0, 126, 0, 255),
 '3': (0, 60, 0, 255),
 '4': (170, 170, 0, 255),
 '5': (170, 128, 0, 255),
 '6': (139, 170, 0, 255),
 '7': (255, 178, 101, 255),
 '8': (0, 217, 0, 255),
 '9': (170, 0, 127, 255),
 '10': (255, 85, 255, 255),
 '11': (255, 85, 127, 255),
 '12': (255, 0, 127, 255),
 '13': (255, 85, 255, 255),
 '14': (170, 255, 255, 255),
 '15': (0, 255, 255, 255),
 '16': (85, 170, 255, 255),
 '17': (226, 151, 0, 255),
 '18': (189, 126, 0, 255),
 '19': (150, 100, 0, 255),
 '20': (162, 236, 177, 255),
 '21': (196, 98, 0, 255),
 '22': (170, 85, 0, 255),
 '23': (109, 54, 0, 255),
 '24': (0, 170, 127, 255),
 '25': (0, 138, 101, 255),
 '26': (0, 89, 65, 255),
 '27': (233, 233, 175, 255),
 '28': (250, 255, 152, 255),
 '29': (0, 0, 127, 255),
 '30': (199, 200, 188, 255),
 '31': (77, 16, 9, 255),
 '123': (58, 117, 0, 255),
 0: (0, 0, 0, 0)}

Write cmap to raster

In [30]:
#if not os.path.exists(destiny_path + layer_name):
#    os.makedirs(destiny_path + layer_name)

In [19]:
#shutil.copy(style_source_path, destiny_path + layer_name)
#shutil.copy(layer_source_path, destiny_path + layer_name)

In [33]:
layer_cmap_source_path = destiny_path + "/" + layer_name + ".geotiff"

In [ ]:
with rasterio.open(layer_source_path) as src:
    arr = src.read(1)
    meta = src.meta
with rasterio.open(layer_cmap_source_path, 'w', **meta) as dst:
    dst.write(arr, indexes=1)
    dst.write_colormap(1,rgb_dict)
    cmap_result = dst.colormap(1)

In [ ]:
print(cmap_result)

References for zipfile:

https://thispointer.com/python-how-to-create-a-zip-archive-from-multiple-files-or-directory/

https://stackoverflow.com/questions/1855095/how-to-create-a-zip-archive-of-a-directory-in-python

In [25]:
import zipfile
from os.path import basename

In [32]:
zip_file = destiny_path + layer_name + ".zip"

In [33]:
zip_file

'/shared_volume/ftp_dir/Test3_National_MAD-Mex_landsat8_lc_2015_32_classes_512_512.zip'

In [36]:
zipf = zipfile.ZipFile(zip_file, 'w', zipfile.ZIP_DEFLATED)
zipf.write(style_source_path, basename(style_source_path))
zipf.write(layer_source_path, basename(layer_cmap_source_path))
zipf.close()

**4) Execute update link for original dataset**

References:


https://github.com/GeoNode/geonode/blob/master/geonode/utils.py#L1668

https://github.com/GeoNode/geonode/blob/89ea8460f23b90c085b33b10b8f0700b9ab506d4/geonode/base/management/commands/set_all_layers_metadata.py#L116

https://github.com/CONABIO/geonode/blob/master/notebooks/3_geonode_metadata_update.ipynb



In `/shared_volume/geonode/geonode/settings.py` remove `GZIP` as its not working. First stop and then up:

```
docker-compose stop

docker-compose up -d django geoserver postgres nginx
```

And remove `GZIP` from `/shared_volume/geonode/geonode/settings.py`

```
DOWNLOAD_FORMATS_RASTER = [
    'JPEG',
    'PDF',
    'PNG',
    'ArcGrid',
    'GeoTIFF',
    'Gtopo30',
    'ImageMosaic',
    'KML',
    'View in Google Earth',
    'Tiles',
    'GML',
    'QGIS layer file (.qlr)',
    'QGIS project file (.qgs)',
    'Zipped All Files'
]
```

```
#being in docker exec -it spcgeonode_django_1 bash

import os
import django

os.chdir("/spcgeonode")

os.environ.setdefault("DJANGO_SETTINGS_MODULE", "geonode.local_settings")

django.setup()

from geonode.base.models import Link

from geonode.layers.models import Layer

layer = Layer.objects.all()[1]

print(layer.title)

download_url = "the url"

extension = "zip"

name = "Original Dataset" #could be "Raw Data"

mime = "application/octet-stream"

link_type = "original"

Link.objects.update_or_create(resource=layer, url=download_url, defaults=dict(extension=extension, name=name, mime=mime, link_type=link_type))


```


```
#to delete link:

Link.objects.filter(resource=layer,url=download_url).first().delete()

```

Last lines can be executed with:

In [1]:
from docker import APIClient


In [2]:
c = APIClient(base_url='tcp://172.17.0.1:1111')
#c = APIClient(base_url='unix://var/run/docker.sock')


In [3]:
title_layer = "Test3 National land cover map 2015 32 classes 512 512" #this will be input of function

name_tif = "my_tif.zip" #this will be input of function

In [4]:
#string1 = "import os;import django;os.chdir('/spcgeonode');os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'geonode.local_settings');django.setup();from geonode.base.models import Link;from geonode.layers.models import Layer;"
string1 = """import os;
import django;
os.chdir('/spcgeonode');
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'geonode.local_settings');
django.setup();
from geonode.base.models import Link;
from geonode.layers.models import Layer;
"""

In [5]:
string2 = "layer = Layer.objects.filter(title="
                                         
string3 = ").first();"

In [6]:
download_url = "ftp://geonode.conabio.gob.mx/pub/" + name_tif

string4 = """Link.objects.update_or_create(resource=layer, url=download_url, 
defaults=dict(extension=extension, name=name_link, mime=mime, link_type=link_type))
"""

In [7]:
string = "".join([string1, 
                  "title_layer = \'", title_layer, "\';",
                  "name_tif = \'", name_tif, "\';",
                  string2, "\'",
                  title_layer, "\'", 
                  string3,
                  "download_url = \'", download_url, "\';",
                  "extension = \'zip\';",
                  "name_link = \'ZIP\';", #could be "Raw Data or Original Dataset"
                  "mime = \'application/octet-stream\';",
                  "link_type = \'original\';", 
                  string4])

In [8]:
cmd = "".join(["python -c ", "\"", string, "\""])

In [10]:
ex = c.exec_create(container = 'spcgeonode_django_1', 
                   cmd = cmd)
ex_start = c.exec_start(exec_id=ex) #if arg stream=True in exec_start method, then returns generator

print(ex_start)

<generator object APIClient._read_from_socket.<locals>.<genexpr> at 0x7fe2b5ae2888>


# 3) Small to Medium size vectors: lcc MAD-Mex


## a) Normalize, reproject to wgs84

In [1]:
import geopandas as gpd
from pyproj import Proj

from geonode_conabio.wrappers import reproj_normalize_and_write_small_medium_size_vector

In [2]:
direc = '/LUSTRE/MADMEX/tnc_data_steffen_thilo/tnc_marismas/'

In [3]:
input_file = direc + '/marismasnacionales_cambiodecoberturadesuelo_2011-2019_final_20190925.gpkg'
output_filename = direc + '/marismasnacionales_cambiodecoberturadesuelo_2011-2019_final_20190925_using_geopandas.shp'
layer = 'marismasnacionales_cambiodecoberturadesuelo_2011-2019_final_20190925_using_geopandas'

In [4]:
name_attribute1 = "nombre01t1"
name_attribute2 = "nombre01t2"
name_attribute3 = "cambio01"

In [5]:
list_name_attributes = [name_attribute1, name_attribute2,
                        name_attribute3
                       ]

**Read and call wrapper**

In [6]:
gdf = gpd.read_file(input_file)
#dropNA's
gdf.dropna(subset=list_name_attributes, inplace=True)


In [7]:
#check geographic proj
crs_src = gdf.crs
src_crs = crs_src.to_string()
proj_src_crs = Proj(src_crs)
if not proj_src_crs.crs.is_geographic:
    reproj_normalize_and_write_small_medium_size_vector(gdf, 
                                                        list_name_attributes,
                                                        layer, output_filename,
                                                        is_geographic=False)
else:
    reproj_normalize_and_write_small_medium_size_vector(gdf, 
                                                        list_name_attributes,
                                                        layer, output_filename)

## b) Register vector layer in geonode via `importlayers`

**1) Importlayers cmd**

In [15]:
region = "".join(["\"Aguascalientes, Mexico, North America, Latin America\""])
name = "\"madmex_sentinel2_aguascalientes_2017_2018_lcc_2\""
title = "\"madmex_sentinel2_aguascalientes_2017_2018_lcc_2\""
abstract = "\"Sentinel2 MAD-Mex land cover change\""
kw = "\"MAD-Mex, Sentinel2, GeoTIFF, WCS\""


In [16]:
from utils_docker import import_layers_via_docker

In [17]:
result_import = import_layers_via_docker(region, name, title,
                                         abstract, kw,
                                         output_filename
                                         )

In [19]:
#print(result_import)

**2) Make sure you are able to download it and see thumbnail. If not click to button refresh attributes and statistics for the layer in geonode. For thumbnail increase nginx conf proxy_read_timeout parameter.**

# Towards python3 package

Following

https://python-packaging.readthedocs.io/en/latest/minimal.html